In [1]:
import numpy as np
import tensorflow as tf

In [2]:
%%time
X_train = np.load("../data/array_data_mini/X_train.npz")["arr_0"]
y_train = np.load("../data/array_data_mini/y_train.npz")["arr_0"]
X_val = np.load("../data/array_data_mini/X_val.npz")["arr_0"]
y_val = np.load("../data/array_data_mini/y_val.npz")["arr_0"]

CPU times: user 1.64 s, sys: 202 ms, total: 1.84 s
Wall time: 1.87 s


In [3]:
print(X_train.shape)
print(y_train.shape)
print(X_val.shape)
print(y_val.shape)

(289000, 128, 1)
(289000, 128, 1)
(25500, 128, 1)
(25500, 128, 1)


In [4]:
seq_size = (X_train[0].shape[0],1)
n_classes = 6
batch_size = 32

In [5]:
print(seq_size)

(128, 1)


In [6]:
class Gen_Data(tf.keras.utils.Sequence):
    
    # Generate data batches for model
    
    def __init__(self,batch_size,seq_size,X_data,y_data):
        
        # Set required attributes
        self.batch_size = batch_size
        self.t_size = seq_size
        self.X_data = X_data
        self.y_data = y_data
        self.n_batches = len(self.X_data)//self.batch_size
        self.batch_idx = np.array_split(range(len(self.X_data)), self.n_batches)
            
    def __len__(self):

        # Set number of batches per epoch
        return self.n_batches
    
    def __getitem__(self,idx):

        # Fetch one batch   
        batch_X = self.X_data[self.batch_idx[idx]]
        batch_y = self.y_data[self.batch_idx[idx]]
        return batch_X, batch_y      

In [7]:
train_gen = Gen_Data(batch_size, seq_size, X_train, y_train)
val_gen = Gen_Data(batch_size, seq_size, X_val, y_val)

In [8]:
def get_model(seq_size, n_classes):
    inputs = tf.keras.Input(shape=seq_size)

    # entry block
    x = tf.keras.layers.Conv1D(32, 5, strides=2, padding="same")(inputs)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation("relu")(x)
    block_out = x
    
    # downsample blocks
    for f in [64,128,256]:
        
        x = tf.keras.layers.Conv1D(f, 5, strides=1, padding="same")(x)
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.Activation("relu")(x)

        x = tf.keras.layers.Conv1D(f, 5, strides=1, padding="same")(x)
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.Activation("relu")(x)

        x = tf.keras.layers.MaxPooling1D(3, strides=2, padding="same")(x)

        residual = tf.keras.layers.Conv1D(f, 1, strides=2, padding="same")(block_out)
        x = tf.keras.layers.add([x, residual])
        block_out = x   

    # upsample blocks
    for f in [256, 128, 64, 32]:
        
        x = tf.keras.layers.Conv1DTranspose(f,5,padding="same")(x)
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.Activation("relu")(x)        
        
        x = tf.keras.layers.Conv1DTranspose(f,5,padding="same")(x)
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.Activation("relu")(x) 
        
        x = tf.keras.layers.UpSampling1D(2)(x) 
                
        residual = tf.keras.layers.UpSampling1D(2)(block_out)
        residual = tf.keras.layers.Conv1D(f, 1, padding="same")(residual)
        x = tf.keras.layers.add([x, residual])
        block_out = x              
        
    # Add a per-pixel classification layer
    outputs = tf.keras.layers.Conv1D(n_classes, 3, activation="softmax", padding="same")(x)    
    
    # Define the model
    model = tf.keras.Model(inputs, outputs)
    return model    

In [9]:
tf.keras.backend.clear_session()

In [10]:
model = get_model(seq_size,n_classes)

In [11]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 128, 1)]     0           []                               
                                                                                                  
 conv1d (Conv1D)                (None, 64, 32)       192         ['input_1[0][0]']                
                                                                                                  
 batch_normalization (BatchNorm  (None, 64, 32)      128         ['conv1d[0][0]']                 
 alization)                                                                                       
                                                                                                  
 activation (Activation)        (None, 64, 32)       0           ['batch_normalization[0][0]']

In [12]:
loss = tf.keras.losses.SparseCategoricalCrossentropy()
opt = tf.keras.optimizers.Adam()

In [13]:
model.compile(optimizer=opt,loss=loss)

In [14]:
epochs = 5
model.fit(train_gen, epochs=epochs, validation_data=val_gen)

Epoch 1/5
9031/9031 [==============================] - 1429s 158ms/step - loss: 0.2185 - val_loss: 0.1385
Epoch 2/5
9031/9031 [==============================] - 1418s 157ms/step - loss: 0.1137 - val_loss: 0.1081
Epoch 3/5
9031/9031 [==============================] - 1437s 159ms/step - loss: 0.0859 - val_loss: 0.0842
Epoch 4/5
9031/9031 [==============================] - 1447s 160ms/step - loss: 0.0717 - val_loss: 0.0760
Epoch 5/5
9031/9031 [==============================] - 1418s 157ms/step - loss: 0.0622 - val_loss: 0.0643


In [16]:
model.save("../models/u_net_mini/")

INFO:tensorflow:Assets written to: ../models/u_net_mini/assets


/opt/anaconda3/lib/python3.7/site-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)
/opt/anaconda3/lib/python3.7/site-packages/keras/saving/saved_model/layer_serialization.py:112: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  return generic_utils.serialize_keras_object(obj)
